<a href="https://colab.research.google.com/github/Jerrywang959/Julia_study/blob/master/bank_loss_viajulia_ipynb%EF%BC%88%E5%89%AF%E6%9C%AC%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安装CUDA和julia-1.4.0

In [1]:
!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.4/julia-1.4.0-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg" up; add IJulia CuArrays Flux FileIO Colors BenchmarkTools CUDAdrv CUDAnative CUDAapi CSV Random;  precompile"'

    Cloning default registries into `~/.julia`
    Cloning registry from "https://github.com/JuliaRegistries/General.git"
      Added registry `General` to `~/.julia/registries/General`
   Updating registry at `~/.julia/registries/General`
   Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
  Installed IJulia ─────────────────────── v1.21.1
  Installed BenchmarkTools ─────────────── v0.5.0
  Installed FileIO ─────────────────────── v1.2.4
  Installed Colors ─────────────────────── v0.12.0
  Installed CUDAdrv ────────────────────── v6.2.2
  Installed Flux ───────────────────────── v0.8.3
  Installed CuArrays ───────────────────── v2.0.1
  Installed CUDAnative ─────────────────── v3.0.3
  Installed DataAPI ────────────────────── v1.1.0
  Installed Missings ───────────────────── v0.4.3
  Installed Reexpor

# 装包专栏


In [0]:
] add Statistics

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
  [10745b16] + Statistics 
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


# 正式的julia运行

加载包

In [0]:
using Flux, Statistics,CSV,DataFrames,Random, CUDAapi,CuArrays

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1260


数据读取和预处理

In [0]:
train_raw=DataFrame(CSV.read("./select-data.csv"))
test_raw=DataFrame(CSV.read("./scalar-test.csv"))

#打乱数据
rnd=Random.shuffle(collect(1:length(train_raw[!,1])))
train_raw=train_raw[rnd,:]
rnd=Random.shuffle(collect(1:length(test_raw[!,1])))
test_raw=test_raw[rnd,:]

#数据分类
train=Array(train_raw[!,2:11])'  |> gpu
target=Array(train_raw[!,12])
test=Array(test_raw[!,2:11])  |> gpu
test_target=Array(test_raw[!,12]);


#One-Hot编码
target=Flux.onehotbatch(target,0:1)     |> gpu

test_target=Flux.onehotbatch(test_target,0:1)  

2×1000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  0  1  1  1  1  …  1  1  0  1  1  1  0  0  0  0  1  1
 0  0  0  0  0  0  0  0  1  0  0  0  0     0  0  1  0  0  0  1  1  1  1  0  0

定义网络结构

In [0]:
m = Chain(
  Dense(10, 256, relu),
  Dense(256, 256,relu),
  Dense(256, 2),
  softmax)    |> gpu

Chain(Dense(10, 256, relu), Dense(256, 256, relu), Dense(256, 2), softmax)

定义交叉熵

In [0]:
loss(x, y) = crossentropy(m(x), y)

loss (generic function with 1 method)

定义损失

In [0]:
accuracy(x, y) = mean(Flux.onecold(cpu(m(x))) .== Flux.onecold(cpu(y)))

accuracy (generic function with 1 method)

In [0]:
dataset = Iterators.repeated((train, target), 200)
evalcb = () -> @show(loss(train, target))
opt = ADAM()
Flux.train!(loss, params(m), dataset, opt, cb = Flux.throttle(evalcb, 10))
@show accuracy(train, target)

loss(train, target) = 0.3164389f0
accuracy(train, target) = 0.8753221254141612


0.8753221254141612

MethodError: ignored

优化方式

MethodError: ignored

In [0]:
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
using CUDAapi
if has_cuda()
    @info "CUDA is on"
    import CuArrays
    CuArrays.allowscalar(false)
end


# Classify MNIST digits with a simple multi-layer-perceptron

imgs = MNIST.images()
# Stack images into one large batch
X = hcat(float.(reshape.(imgs, :))...) |> gpu

labels = MNIST.labels()
# One-hot-encode the labels
Y = onehotbatch(labels, 0:9) |> gpu

m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax) |> gpu

loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(cpu(m(x))) .== onecold(cpu(y)))

dataset = repeated((X, Y), 200)
evalcb = () -> @show(loss(X, Y))
opt = ADAM()

Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 10))

@show accuracy(X, Y)

# Test set accuracy
tX = hcat(float.(reshape.(MNIST.images(:test), :))...) |> gpu
tY = onehotbatch(MNIST.labels(:test), 0:9) |> gpu

@show accuracy(tX, tY)

┌ Info: CUDA is on
└ @ Main In[4]:6
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /root/.julia/packages/Flux/oX9Pi/src/data/mnist.jl:24
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /root/.julia/packages/Flux/oX9Pi/src/data/mnist.jl:24
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /root/.julia/packages/Flux/oX9Pi/src/data/mnist.jl:24
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /root/.julia/packages/Flux/oX9Pi/src/data/mnist.jl:24
┌ Info: Building the CUDAnative run-time library for your sm_60 device, this might take a while...
└ @ CUDAnative /root/.julia/packages/CUDAnative/hfulr/src/compiler/rtlib.jl:188


loss(X, Y) = 2.3505874f0
accuracy(X, Y) = 0.9245
accuracy(tX, tY) = 0.9258


0.9258